# Quickstart for data modeling and power-ops-sdk for cogshop runs
NOTE: The "recommended" way of running CogShop is as follows:
1. Static model data and mappings for time-dependent data like time series represented by `ShopModel`
2. Any variations on the base configuration (like different price scenarios) represented by `ShopScenario` instances 
3. Generate a complete `ShopCase` with time series datapoints based on a `ShopScenario` instance plus `startTime` and `endTime` through the shop_trigger Cognite function.

This tutorial covers a different set-up: Triggering (Cog)Shop for a pre-generated, complete Shop "case" (as a yaml file)

## Preparation

The main entry point for the SDK is the `PowerOpsClient`

To get started, we assume you have installed the `cognite-power-ops` SDK and that all configurations have been set up in Cognite Data Fusion (CDF).

Furthermore, it is assumed that you have set up one or two `toml` files with the credentials for connecting to Cognite Data Fusion and settings for this SDK.


### Interactive Login
This only requires one `toml` file looking like this

`settings.toml`

```toml
[cognite]
  login_flow = "interactive"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"

```

### Client Credentials Login

For this case the you can use two toml files to separate the secrets from the regular settings.


`settings.toml`

```toml
[cognite]
  login_flow = "client_credentials"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"
```
and the `.secrets.toml`

```toml
[cognite]
  client_secret = "<client-secret>"

```

Values in `.secrets.toml` will overwrite those in `settings.toml`




In [2]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops._version import __version__
from cognite.powerops import PowerOpsClient


os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

WRITE_DATA_SET = powerops.datasets.write_dataset_id

print(__version__)  # Print the version of the package

0.101.2


## STEPS:
1. Upload a yaml file that can be used by Shop, and return the external id
2. Set up a ShopCase, using the SDK
3. Write (upsert) the ShopCase instance to CDF, and verify that it was created using the SDK
4. Trigger a shop execution of that shop case
5. List the ShopResult referencing the ShopCase 


### Step 1: Upload a file 

In the case that the file already exists, this step can be skipped. 

We need the external id for constructing the `ShopCase`

In [3]:
from pathlib import Path


def upload_file(file_name: str) -> str:
    """Upload a file to CDF and return the external id of the file"""
    file_path = (Path("example_case_files") / file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id="example_stavanger_case_file",
        name=file_name,
        data_set_id=WRITE_DATA_SET,
        mime_type="application/yaml",
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id


# NB! The case in this example is minimal and just for demo purposes.
my_file_reference = upload_file(file_name="a_example_stavanger_with_commands.yaml")
my_file_reference

'example_stavanger_case_file'

### Step 2 Preparing a ShopCase

`prepare_shop_case` creates a `Write` object can be written to CDF. 

We use the generated SDK (`_generated`) as they are a direct reflection of the Data Models on CDF. Here the data classes end in `Write` since we are going to use the to send data to CDF. They all have a corresponding class used for retrieving data from CDF.

In [4]:
import datetime

# Unique identifier for the case, if not provided, an external id will be generated
EXAMPLE_CASE_EXTERNAL_ID = "example_case_external_id"

#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
file_tuple = (my_file_reference, "stavanger_case_file", False, "")


my_shop_case = powerops.cogshop.prepare_shop_case(
    shop_file_list=[file_tuple],
    shop_version="15.6.1.0",
    start_time=datetime.datetime(2024, 5, 31, 22),
    end_time=datetime.datetime(2024, 6, 2, 22),
    model_name="my_model",
    scenario_name="my_scenario",
    case_external_id=EXAMPLE_CASE_EXTERNAL_ID,  # can be auto generated if set to None/not provided
)
my_shop_case

,value
space,power_ops_instances
external_id,example_case_external_id
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2024-05-31 22:00:00
end_time,2024-06-02 22:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 3: Upload the ShopCase

Note: The upload is an `upsert` -- so if there are overlaps, the existing properties will merge  


In [5]:
powerops.cogshop.write_shop_case(my_shop_case)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='example_case_external_id', version=2) at 0x7fda00892bd0>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:55ebd767bbb948c8a0457db9631bb41d', version=1) at 0x7fda00891cd0>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:792fe2104d094fb187a52f7a9142c3ca', version=1) at 0x7fda00891490>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:80716824a3af45f0a838431341f734c9', version=1) at 0x7fda01a43790>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='example_case_external_id:shopfile:55ebd767bbb948c8a0457db9631bb41d', version=1) at 0x7fda01a436d0>], time_series=[])

Notice that the retrieved case is not suffixes with a `Write` 

(However, it is possible to convert it to a `Write` object, and upsert again to make changes)

In [6]:
from cognite.powerops.client._generated.v1.data_classes import ShopCase

# NB! This step is not necessary, if the upsert was successful we know the case is in CDF
retrieved_shop_case: ShopCase = powerops.cogshop.retrieve_shop_case(
    my_shop_case.external_id
)
retrieved_shop_case

,value
space,power_ops_instances
external_id,example_case_external_id
data_record,"{'version': 2, 'last_updated_time': 2024-11-08..."
node_type,None
scenario,shopscenario:792fe2104d094fb187a52f7a9142c3ca
start_time,2024-05-31 22:00:00+00:00
end_time,2024-06-02 22:00:00+00:00
shop_files,"[shopfile:50cfb842a5494433909b35c6fba19a4c, sh..."


Alternately, use Graph QL to retrieve the Shop Case instance. 
Under the hood, this uses a different method to look for the case so the returned data has a slightly different format

In [7]:
graph_ql_shop_case = powerops.cogshop.retrieve_shop_case_graphql(
    my_shop_case.external_id
)
graph_ql_shop_case

,value
space,power_ops_instances
external_id,example_case_external_id
data_record,"{'version': 0, 'last_updated_time': 2024-11-08..."
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2024-05-31 22:00:00+00:00
end_time,2024-06-02 22:00:00+00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 4: Trigger the case

We will here directly use the new `trigger_shop_case` method on the `powerops.cogshop` client

In [8]:
powerops.cogshop.trigger_shop_case(my_shop_case.external_id)

### Step 5: View the ShopResult generated 

Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned. 

Note that since it is possible to run trigger cogshop multiple times for the same case, `ShopResults` are returned as lists. 
  

Explanations for (some) of the fields of the `ShopResult`: 

* External id: The external id of the `ShopResult`
* Case: The external id of the `ShopCase` that the instance belongs to
* Objective value: The objective value of the Shop execution
* Pre run: A file reference to the pre run yaml file
* Post run: A file reference to the post run yaml file
* Messages: A file reference to the logs generated by Shop
* Cplex logs: A file reference to the cplex logs generated by Shop
* Data record: The data record of the instance, contains the keys `last_updated_time` and `created_time`



#### 1. Using `list_shop_results_for_case`

Note that this return type is a list from `data_classes`, which means it can be easily converted to a pandas dataframe

In [9]:
from cognite.powerops.client._generated.v1.data_classes import ShopResultList

result_list: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=my_shop_case.external_id
)
result_list

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,power_ops_instances,shop_result__7c9091a7-5286-4dce-a4f9-edb925374a5d,example_case_external_id,"{'total': -19927822739.279236, 'load_value': 0...",example_case_file_external_id,POWEROPS_SHOP_post-run-7c9091a7-5286-4dce-a4f9...,POWEROPS_SHOP_shop-7c9091a7-5286-4dce-a4f9-edb...,POWEROPS_SHOP_cplex-7c9091a7-5286-4dce-a4f9-ed...,[shop_penalty_report_dummy_scenario_2023-09-14...,None,None,"{'version': 1, 'last_updated_time': 2024-10-14..."


#### 2. Using Graph QL and `list_shop_result_graphql`

Note that this return type is an ordinary python `list` of `ShopResult`s, meaning it cannot seamlessly be converted to a dataframe  

In [10]:
from cognite.powerops.client._generated.v1.data_classes import ShopResult


result_list_ql: list[ShopResult] = powerops.cogshop.list_shop_result_graphql(
    case_external_id=my_shop_case.external_id, limit=1
)
result_list_ql

[ShopResult(space='power_ops_instances', external_id='shop_result__7c9091a7-5286-4dce-a4f9-edb925374a5d', data_record=DataRecord(version=0, last_updated_time=datetime.datetime(2024, 10, 14, 10, 42, 52, 768000, tzinfo=TzInfo(UTC)), created_time=datetime.datetime(2024, 10, 14, 10, 42, 52, 768000, tzinfo=TzInfo(UTC)), deleted_time=None), node_type=None, objective_value=None, pre_run='example_case_file_external_id', post_run='POWEROPS_SHOP_post-run-7c9091a7-5286-4dce-a4f9-edb925374a5d', messages='POWEROPS_SHOP_shop-7c9091a7-5286-4dce-a4f9-edb925374a5d', cplex_logs='POWEROPS_SHOP_cplex-7c9091a7-5286-4dce-a4f9-edb925374a5d')]

#### 3. Using the external ID of a ShopResult to retrieve

If we know a specific `ShopResult`is of interest, we can use its external id to retrieve it and inspect it  

In [11]:
shop_result = powerops.cogshop.retrieve_shop_result(result_list_ql[0].external_id)

In [12]:
shop_result

,value
space,power_ops_instances
external_id,shop_result__7c9091a7-5286-4dce-a4f9-edb925374a5d
data_record,"{'version': 1, 'last_updated_time': 2024-10-14..."
node_type,None
case,example_case_external_id
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,example_case_file_external_id
post_run,POWEROPS_SHOP_post-run-7c9091a7-5286-4dce-a4f9...
messages,POWEROPS_SHOP_shop-7c9091a7-5286-4dce-a4f9-edb...
cplex_logs,POWEROPS_SHOP_cplex-7c9091a7-5286-4dce-a4f9-ed...


## Cleaning up the instances that were created

Instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [14]:
all_external_ids = [
    shop_result.external_id,
    my_shop_case.external_id,
    my_shop_case.shop_files[0].external_id,
    my_shop_case.scenario.external_id,
    my_shop_case.scenario.model.external_id,
]
all_external_ids

['shop_result__7c9091a7-5286-4dce-a4f9-edb925374a5d',
 'example_case_external_id',
 'shopfile:55ebd767bbb948c8a0457db9631bb41d',
 'shopscenario:792fe2104d094fb187a52f7a9142c3ca',
 'shopmodel:80716824a3af45f0a838431341f734c9']

In [15]:
powerops.v1.delete(external_id=all_external_ids)

InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='shop_result__7c9091a7-5286-4dce-a4f9-edb925374a5d'), NodeId(space='power_ops_instances', external_id='example_case_external_id'), NodeId(space='power_ops_instances', external_id='shopfile:55ebd767bbb948c8a0457db9631bb41d'), NodeId(space='power_ops_instances', external_id='shopscenario:792fe2104d094fb187a52f7a9142c3ca'), NodeId(space='power_ops_instances', external_id='shopmodel:80716824a3af45f0a838431341f734c9')], edges=[])